# Touristic Tour Recommendation Application
This notebook outlines the steps involved in creating an algorithm that generates a one-week itinerary for tourists in Algeria. The itinerary is optimized based on user preferences, proximity, and travel costs. Various search techniques, including **Uninformed Search Algorithms**, **A*** and **Hill Climbing**, are employed to create the optimal itinerary.


## Data Collection & Research
We gathered - clean - data about **+200 Algerian tourist attractions**, including the following attributes:
- **Attraction Name**
- **Type of Attraction** (museum, nature, beach, etc.)
- **City**
- **Cost** (entry fee)
- **Rating** (user rating)
- **GPS Coordinates** (latitude, longitude)
- **Description** (short description)


In [1]:
import json
from collections import Counter

DATA_PATH = "../Data/attractions.json"

with open(DATA_PATH, "r", encoding="utf-8") as f:
    attractions_data = json.load(f)
    

if not isinstance(attractions_data, list):
    raise ValueError("The JSON file does not contain a list of attractions.")

print("Number of attractions:", len(attractions_data))

# Count attractions per city
city_counts = Counter(attraction.get("city", "Unknown") for attraction in attractions_data)

# Count attractions per category
category_counts = Counter(attraction.get("category", "Unknown") for attraction in attractions_data)

print("\nNumber of attractions per city:")
for city, count in city_counts.items():
    print(f"{city}: {count}")

print("\nNumber of attractions per category:")
for category, count in category_counts.items():
    print(f"{category}: {count}")


Number of attractions: 203

Number of attractions per city:
Algiers: 18
Tipaza: 3
Blida: 1
Médéa: 3
Oran: 16
Tlemcen: 9
Batna: 1
Ghardaïa: 2
Béjaïa: 18
Constantine: 6
Djanet: 7
Sétif: 13
Annaba: 10
Guelma: 3
El Tarf: 7
Tamanrasset: 3
Béchar: 1
Bouira: 1
El Bayadh: 1
Khenchela: 1
Biskra: 4
Timimoun: 3
El Oued: 1
M'Sila: 1
Tizi Ouzou: 21
Beni Abbes: 1
Skikda: 6
Souk Ahras: 6
Tébessa: 3
Oum El-Bouaghi: 5
Jijel: 20
Aïn Témouchent: 2
Boumerdès: 6

Number of attractions per category:
Garden: 3
Museum: 11
Cultural: 22
Historical: 35
Religious: 10
Amusement Park: 5
Port: 2
Shopping Mall: 4
Nature: 70
Lake: 4
Adventure: 1
Resort: 3
Beach: 29
Island: 1
Wildlife Park: 1
Coastal Town: 1
Recreational: 1


## Problem Formulation
### 1. State Representation

Our state representation is a dictionary with the following structure:

```python
state = {
    'current_location': (lat, lon),         # Current GPS coordinates
    'itinerary': [[] for _ in range(7)],    # 7 days of planned attractions (names)
    'curr_day': 0,                          # Current day index (0-6)
    'total_cost': 0,                        # Accumulated cost so far
    'total_time': 0,                        # Total travel time
    'daily_time': [0]*7                     # Time used per day (including visits)
}
```

### 2. Actions

Two possible actions:
1. `('add', attraction)`: Add an attraction to the current day
2. `('next_day',)`: Move to the next day of planning

### 3. Goal Test

The goal is reached when:
- All 7 days have been planned (`curr_day >= 7`)
- Each day has at least one attraction

### 4. Path Cost

The path cost is the cumulative cost of visiting attractions and traveling between them.

### Problem Class
Now, let's define the main Problem class that will encapsulate our tour planning problem

In [2]:
import math
import json
import re
from copy import deepcopy
from typing import List, Dict, Tuple  # Helper library for type hinting
 
class TourPlanningProblem:
    def __init__(self, initial_state: Dict, attractions: List[Dict], 
                 user_prefs: Dict, constraints: Dict):
        """
        Args:
            initial_state: Initial state dictionary.
            attractions: List of attraction dictionaries.
            user_prefs: User preferences dictionary.
            constraints: Problem constraints dictionary.
        """
        self.initial_state = initial_state
        self.attractions = attractions
        self.user_prefs = user_prefs
        self.constraints = constraints
        self.distance_cache = self._build_distance_cache()
        
    def _build_distance_cache(self) -> Dict[Tuple[str, str], float]:
        """Precompute distances between all pairs of attractions."""
        cache = {}
        for a1 in self.attractions:
            for a2 in self.attractions:
                key = (a1['name'], a2['name'])
                cache[key] = self._calculate_distance(a1['gps'], a2['gps'])
        return cache
    
    def _calculate_distance(self, coord1: List[float], coord2: List[float]) -> float:
        """Haversine distance between two GPS coordinates."""
        lat1, lon1 = math.radians(coord1[0]), math.radians(coord1[1])
        lat2, lon2 = math.radians(coord2[0]), math.radians(coord2[1])
        dlat = lat2 - lat1
        dlon = lon2 - lon1
        a = math.sin(dlat/2)**2 + math.cos(lat1) * math.cos(lat2) * math.sin(dlon/2)**2
        c = 2 * math.atan2(math.sqrt(a), math.sqrt(1-a))
        return 6371 * c  # Earth radius in km
    
    def actions(self, state: Dict) -> List[Tuple]:
        """
        Return a list of possible actions from the current state.
        Two types of actions:
          - ('add', <attraction_dict>): add an attraction to the current day.
          - ('next_day',): move to the next day.
        """
        valid_actions = []
        curr_day = state['curr_day']
        
        # If all days are planned, no further actions
        if curr_day >= len(state['itinerary']):
            return []
            
        # If the current day is full, only allow moving to next day
        if len(state['itinerary'][curr_day]) >= self.constraints['max_attractions_per_day']:
            return [('next_day',)]
            
        # Check each attraction for validity
        for attraction in self.attractions:
            if self._is_valid_addition(state, attraction):
                valid_actions.append(('add', attraction))
                
        # Allow 'next_day' if there's at least one attraction in the current day
        if len(state['itinerary'][curr_day]) > 0:
            valid_actions.append(('next_day',))
            
        return valid_actions
    
    def _is_valid_addition(self, state: Dict, attraction: Dict) -> bool:
        """
        Check whether adding this attraction to the current day is valid.
        Ensures no duplicate visits, and that cost and time constraints are respected.
        """
        curr_day = state['curr_day']
        if curr_day >= 7:
            return False

        # Avoid duplicates across all days
        for day_list in state['itinerary']:
            if attraction['name'] in day_list:
                return False

        # Check daily limit of attractions
        if len(state['itinerary'][curr_day]) >= self.constraints['max_attractions_per_day']:
            return False

        # Check that new cost does not exceed budget
        new_cost = state['total_cost'] + self._parse_cost(attraction['cost'])
        if new_cost > self.constraints['max_total_budget']:
            return False

        # Check that adding the attraction doesn't exceed the max daily time
        travel_time = self._estimate_travel_time(state, attraction)
        visit_time = self._parse_duration(attraction['visit_duration'])
        day_time_used = state['daily_time'][curr_day]
        new_day_time = day_time_used + travel_time + visit_time
        if new_day_time > self.constraints['max_daily_time']:
            return False

        return True
    
    def _estimate_travel_time(self, state: Dict, new_attraction: Dict) -> float:
        """
        Estimate travel time from the last visited attraction (or current location)
        to the new attraction.
        Assumes an average speed of 50 km/h.
        """
        curr_day = state['curr_day']
        day_attractions = state['itinerary'][curr_day]
        
        if not day_attractions:
            last_coords = state['current_location']
        else:
            last_att_name = day_attractions[-1]
            last_att = next(a for a in self.attractions if a['name'] == last_att_name)
            last_coords = last_att['gps']
            
        distance = self._calculate_distance(last_coords, new_attraction['gps'])
        return distance / 50  # hours

    def result(self, state: Dict, action: Tuple) -> Dict:
        """
        Return a new state after applying the given action to the current state.
        """
        new_state = deepcopy(state)
        action_type = action[0]
        curr_day = new_state['curr_day']
        
        if action_type == 'add':
            _, attraction = action
            new_state['itinerary'][curr_day].append(attraction['name'])
            new_state['total_cost'] += self._parse_cost(attraction['cost'])
            visit_time = self._parse_duration(attraction['visit_duration'])
            travel_time = self._estimate_travel_time(state, attraction)
            new_state['daily_time'][curr_day] += travel_time + visit_time
            new_state['total_time'] += travel_time  # add travel time to overall total
        elif action_type == 'next_day':
            new_state['curr_day'] += 1
            
        return new_state
    
    def path_cost(self, current_cost: float, state1: Dict, action: Tuple, state2: Dict) -> float:
        """
        Calculate the path cost when taking an action.
        For 'add' actions, we add the attraction cost.
        """
        if action[0] == 'add':
            return current_cost + self._parse_cost(action[1]['cost'])
        return current_cost  
    
    def travel_cost_km(self, gps_a: List[float], gps_b: List[float]) -> float:
        """Set a cost rate of 10 DZD per km traveled."""
        dist = self._calculate_distance(gps_a, gps_b)
        return dist * 10.0

    def is_goal(self, state: Dict) -> bool:
        """
        The goal is reached if:
         1) We have planned all 7 days (curr_day >= 7).
         2) Each day has at least one attraction.
         3) The total cost is within budget.
        """
        if state['curr_day'] < 7:
            return False
        for day_list in state['itinerary']:
            if len(day_list) == 0:
                return False
        if state['total_cost'] > self.constraints['max_total_budget']:
            return False
        return True
    
    def value(self, state: Dict) -> float:
        """Objective function to maximize (satisfaction - penalties)."""
        return self._calculate_satisfaction(state) - self._calculate_penalties(state)
    
    def _calculate_satisfaction(self, state: Dict) -> float:
        """Calculate user satisfaction score (0-100 scale)."""
        score = 0
        preferred_cats = set(self.user_prefs['categories'])
        for day in state['itinerary']:
            for att_name in day:
                attraction = next(a for a in self.attractions if a['name'] == att_name)
                if attraction['category'] in preferred_cats:
                    score += 10 * attraction['rating']
                else:
                    score += 5 * attraction['rating']
        max_possible = 10 * 5 * 7 * self.constraints['max_attractions_per_day']
        return (score / max_possible) * 100
    
    def _calculate_penalties(self, state: Dict) -> float:
        """Calculate penalty terms for constraints."""
        cost_penalty = (state['total_cost'] / self.constraints['max_total_budget']) * 50
        time_penalty = (sum(state['daily_time']) / (7 * self.constraints['max_daily_time'])) * 30
        return cost_penalty + time_penalty
    

    @staticmethod
    def _parse_cost(cost_str: str) -> float:
        """
        Convert cost field (e.g., "Free", "400 DZD", "Unknown", "Variable") to a numeric value.
        - "Free" -> 0
        - Otherwise, return the first integer found or default to 300.
        """
        cost_str_lower = cost_str.lower()
        if "free" in cost_str_lower:
            return 0.0
        match = re.search(r"(\d+)", cost_str)
        if match:
            return float(match.group(1))
        return 300.0
    
    @staticmethod
    def _parse_duration(duration_str: str) -> float:
        """
        Convert a duration string (e.g., "1-2 hours", "3 hours") to a numeric estimate (hours).
        """
        duration_str = duration_str.lower()
        match_range = re.match(r"(\d+)-(\d+)\s*hours", duration_str)
        if match_range:
            low = float(match_range.group(1))
            high = float(match_range.group(2))
            return (low + high) / 2.0
        match_single = re.match(r"(\d+)\s*hours", duration_str)
        if match_single:
            return float(match_single.group(1))
        return 2.0  # Default

### Node Class
First, let's define the Node class which will represent states in our search space:
A state represents a partial itinerary

In [3]:
class Node:
    def __init__(self, state: Dict, parent: 'Node' = None, 
                 action: Tuple = None, path_cost: float = 0):
        """
        Args:
            state: The current state dictionary
            parent: Parent node
            action: Action that led to this node
            path_cost: Cumulative cost to reach this node
        """
        self.state = deepcopy(state)
        self.parent = parent
        self.action = action
        self.path_cost = path_cost
        self.depth = parent.depth + 1 if parent else 0
        self.value = None  # Will store heuristic/objective value
        
    def __lt__(self, other: 'Node') -> bool:
        """
        For priority queue ordering if needed (like in A*).
        If self.value is None, defaults to 0 for comparison.
        """
        return self.value < other.value if self.value is not None else False
        
    def expand(self, problem: TourPlanningProblem) -> List['Node']:
        """Generate all child nodes reachable from this node"""
        child_nodes = []
        for action in problem.actions(self.state):
            next_state = problem.result(self.state, action)
            child_node = Node(
                state=next_state,
                parent=self,
                action=action,
                path_cost=problem.path_cost(self.path_cost, self.state, action, next_state)
            )
            child_nodes.append(child_node)
        return child_nodes
        
    def path(self) -> List['Node']:
        """Return the path from root to this node"""
        node, path = self, []
        while node:
            path.append(node)
            node = node.parent
        return list(reversed(path))
        
    def __repr__(self) -> str:
        return (f"Node(day={self.state['curr_day']}, "
                f"cost={self.state['total_cost']}, "
                f"value={self.value})")
    
    def __eq__(self, other: 'Node') -> bool:
        """
        Equality check based on the state's itinerary and day.
        """
        return isinstance(other, Node) and self.state == other.state

    def __hash__(self) -> int:
        """
        Hash based on the itinerary's arrangement for use in sets/dicts.
        """
        # We'll hash a tuple of (current_day, tuple of each day’s attractions).
        # This avoids collisions from dict ordering
        day_tuples = tuple(tuple(day) for day in self.state['itinerary'])
        return hash((self.state['curr_day'], day_tuples))

## Helper Functions

In [4]:
import math
from typing import List, Tuple, Dict
import json
def load_attractions(json_file: str) -> List[Dict]:
    """Load attractions from JSON file"""
    with open(json_file) as f:
        return json.load(f)

def create_initial_state(start_location: Tuple[float, float], user_prefs: Dict) -> Dict:
    """Create initial state dictionary"""
    return {
        'current_location': start_location,
        'itinerary': [[] for _ in range(7)],
        'curr_day': 0,
        'total_cost': 0.0,
        'total_time': 0.0,
        'daily_time': [0.0]*7,
        'preferences': user_prefs
    }

def haversine_distance(coord1: Tuple[float, float], coord2: Tuple[float, float]) -> float:
    R = 6371  # km
    lat1, lon1 = math.radians(coord1[0]), math.radians(coord1[1])
    lat2, lon2 = math.radians(coord2[0]), math.radians(coord2[1])
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = math.sin(dlat/2)**2 + math.cos(lat1) * math.cos(lat2) * math.sin(dlon/2)**2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1-a))
    return R * c

def build_distance_matrix(attractions: List[Dict]) -> Dict[Tuple[str, str], float]:
    """Create distance lookup table for all attraction pairs"""
    matrix = {}
    for a1 in attractions:
        for a2 in attractions:
            matrix[(a1['name'], a2['name'])] = haversine_distance(a1['gps'], a2['gps'])
    return matrix

def estimate_travel_time(distance_km: float, 
                        transport_mode: str = 'car') -> float:
    """Convert distance to estimated travel time in hours"""
    speeds = {'car': 50, 'bus': 40, 'walking': 5}
    return distance_km / speeds.get(transport_mode, 50)

def calculate_day_time(itinerary_day: List[str], 
                     attractions: List[Dict],
                     distance_matrix: Dict) -> float:
    """Calculate total time for a single day's itinerary"""
    total_time = 0
    for i in range(len(itinerary_day)):
        if i > 0:
            prev_att = next(a for a in attractions if a['name'] == itinerary_day[i-1])
            curr_att = next(a for a in attractions if a['name'] == itinerary_day[i])
            distance = distance_matrix[(prev_att['name'], curr_att['name'])]
            total_time += estimate_travel_time(distance)
        # Note: to be adjusted if we need to look up the visit duration for the current attraction
        total_time += TourPlanningProblem._parse_duration(curr_att['visit_duration'])
    return total_time

def calculate_total_cost(itinerary: List[List[str]],
                       attractions: List[Dict]) -> float:
    """Calculate total cost of itinerary"""
    return sum(TourPlanningProblem._parse_cost(a['cost'])
              for day in itinerary
              for att_name in day
              for a in attractions if a['name'] == att_name)

def estimate_hotel_costs(hotel_standard: Tuple[int, int],
                        num_nights: int = 7) -> float:
    """Estimate hotel costs based on preferred star rating"""
    min_stars, max_stars = hotel_standard
    avg_stars = (min_stars + max_stars) / 2
    # Assuming 3000 DZD per night per star
    return avg_stars * 3000 * num_nights

## User Preferences Simulation 
The application will take inputs (from the website interface) such as:
- **Starting location**: Algiers
- **Preferred attractions**: Museums, Historical Sites
- **Budget**: 5000 DZD
- **Hotel rating**: 3 stars


## Search Algorithm Implementations

In this project, we use a combination of **uninformed search algorithms** (BFS, DFS) and **informed search algorithms** (A\* Search, Hill Climbing) to generate the optimal itinerary based on cost, proximity, and user preferences.

Each search algorithm has its strengths and weaknesses. In this section, we will implement and compare their performance in solving the itinerary optimization problem.


### BFS
Explores all paths at the current depth before moving on to the next level, ensuring the shortest path is found if the graph is unweighted.


In [5]:
from collections import deque

def BFS(problem, max_iterations=10000):
    """
    Perform a BFS to find the shortest path from start to goal.

    Args:
    - problem: TourPlanningProblem instance
    - max_iterations: Maximum number of iterations to prevent infinite loops

    Returns:
    - Solution node or None
    """
    frontier = deque()
    initial_node = Node(problem.initial_state)
    frontier.append(initial_node)

    if problem.is_goal(initial_node.state):
        return initial_node

    explored = set()
    frontier_states = set()
    iterations = 0

    while frontier and iterations < max_iterations:
        iterations += 1
        node = frontier.popleft()  # Dequeue a node
        
        state_hashable = hashh(node.state)
        if state_hashable in explored:
            continue
        
        explored.add(state_hashable)

        for child in node.expand(problem):
            # Convert child state to hashable
            child_hashable = hashh(child.state)
            
            
            if child_hashable not in explored and child_hashable not in frontier_states:
                
                if problem.is_goal(child.state):
                    print(f"Solution found after {iterations} iterations")
                    return child
                
                frontier.append(child)
                frontier_states.add(child_hashable)

    print(f"Stopped after {iterations} iterations")#iterations used only for testing will be removed later
    return None

def hashh(state):
    """
    Recursively converts any unhashable types in a state to hashable types.
     
    Args:
        state: The state to be converted to a hashable type.
     
    Returns:
        A hashable representation of the state.
    """
    if isinstance(state, dict):
        return tuple(sorted((k, hashh(v)) for k, v in state.items()))
    elif isinstance(state, list):
        return tuple(hashh(v) for v in state)
    elif isinstance(state, set):
        return tuple(sorted(hashh(v) for v in state))
    return state  # If already hashable (string, int, tuple)

### BFS Example


In [6]:
def run_BFS():
    constraints = {
        'max_total_budget': 5000.0,
        'max_daily_time': 7.0,
        'max_attractions_per_day': 3
    }

    user_prefs = {
        'categories': ['Museum', 'Historical']
    }

    initial_state = {
        'current_location': (35.6911, -0.6417),  # e.g., Oran coordinates
        'itinerary': [[] for _ in range(7)],
        'curr_day': 0,
        'total_cost': 0.0,
        'total_time': 0.0,
        'daily_time': [0.0] * 7,
        'preferences': user_prefs
    }

    problem = TourPlanningProblem(
        initial_state=initial_state,
        attractions=attractions_data,
        user_prefs=user_prefs,
        constraints=constraints
    )
    solution_node = BFS(problem,10000000)
    if solution_node is None:
        print("BFS: No solution found within the depth limit.")
    else:
        print("BFS found a solution!\n")
        path_nodes = solution_node.path()
        final_state = path_nodes[-1].state

        for d, day_list in enumerate(final_state['itinerary']):
            print(f"Day {d+1}: {day_list}")

        print(f"\nTotal cost: {final_state['total_cost']}")
        print("Daily time usage:")
        for i, t in enumerate(final_state['daily_time'], 1):
            print(f"Day {i}: {t:.2f} hours")
        print(f"Days used: {final_state['curr_day']}")


if __name__ == "__main__":
    run_BFS()

KeyboardInterrupt: 

### DFS
Explores each path as deep as possible before backtracking, often used when exploring solutions in a depth-first manner.

In [7]:
def depth_limited_search(problem: TourPlanningProblem, limit: int):
    """
    A Depth-Limited Search on the TourPlanningProblem.

    :param problem: An instance of the TourPlanningProblem.
    :param limit: Maximum depth (or 'depth limit') for the search.
    :return: The first solution Node that satisfies problem.is_goal(...) or None if no solution is found
             within the specified depth limit.
    """
    # We'll define a recursive inner function that carries the explored set and the current depth as we go deeper.
    def recursive_dls(node: Node, depth: int, explored):
        if problem.is_goal(node.state):
            return node

        if depth == limit:
            return None  # We've reached the depth limit, stop going deeper.

        explored.add(node)

        # Expand the node
        for child in node.expand(problem):
            if child not in explored:
                result = recursive_dls(child, depth + 1, explored)
                if result is not None:
                    return result
        return None

    initial_node = Node(problem.initial_state)
    
    if problem.is_goal(initial_node.state):
        return initial_node

    explored_set = set()
    
    return recursive_dls(initial_node, 0, explored_set)


def depth_first_search(problem: TourPlanningProblem, max_depth: int = None):
    """
    Depth-First Search for the TourPlanningProblem.
    If max_depth is None, we use a very large limit (effectively 'no limit').
    """
    if max_depth is None:
        max_depth = 999999  # effectively no limit

    return depth_limited_search(problem, max_depth)

def iterative_deepening_search(problem: TourPlanningProblem, max_depth: int = 30):
    """
    Iterative Deepening DFS: Repeatedly run depth-limited DFS up to increasing depth limits.
    
    :param problem: An instance of TourPlanningProblem.
    :param max_depth: Maximum depth up to which we'll attempt searches.
    :return: The first solution Node found, or None if no solution is found up to max_depth.
    """
    for depth in range(max_depth + 1):
        result = depth_limited_search(problem, depth)
        if result is not None:
            return result
    return None




### Example Executing DFS

In [8]:
def run_dfs_demo():
    # Define constraints
    constraints = {
        'max_total_budget': 5000.0,
        'max_daily_time': 7.0,
        'max_attractions_per_day': 3
    }
    
    user_prefs = {
        'categories': ['Museum', 'Historical']
    }
    
    initial_state = {
        'current_location': (35.6911, -0.6417),  # e.g., Oran coordinates
        'itinerary': [[] for _ in range(7)],
        'curr_day': 0,
        'total_cost': 0.0,
        'total_time': 0.0,
        'daily_time': [0.0] * 7,
        'preferences': user_prefs
    }
    
    problem = TourPlanningProblem(
        initial_state=initial_state,
        attractions=attractions_data,
        user_prefs=user_prefs,
        constraints=constraints
    )
    
    # Run DFS with a depth limit; alternatively, we can use iterative_deepening_search(problem, max_depth=40)
    solution_node = depth_first_search(problem, max_depth=40)
    
    if solution_node is None:
        print("DFS: No solution found within the depth limit.")
    else:
        print("DFS found a solution!\n")
        path_nodes = solution_node.path()
        final_state = path_nodes[-1].state
        
        for d, day_list in enumerate(final_state['itinerary']):
            print(f"Day {d+1}: {day_list}")
        
        print(f"\nTotal cost: {final_state['total_cost']}")
        print("Daily time usage:")
        for i, t in enumerate(final_state['daily_time'], 1):
            print(f"Day {i}: {t:.2f} hours")
        print(f"Days used: {final_state['curr_day']}")

if __name__ == "__main__":
    run_dfs_demo()

DFS found a solution!

Day 1: ['Fort Santa Cruz', "Bey's Palace", 'Ahmed Zabana National Museum']
Day 2: ['Cathedral of the Sacred Heart of Oran', 'Oran Sea Front', 'Canastel Forest']
Day 3: ['Regional Theater of Oran', 'Centre Commercial et de Loisirs Es-Senia', 'Abdelhamid Ben Badis Mosque']
Day 4: ['AZ Mall Grand Oran', "Arènes d'Oran", 'Disco Maghreb']
Day 5: ['Sebkha of Oran']
Day 6: ['Great Mosque of Tlemcen', 'Al Mechouar Palace']
Day 7: ['Mansourah Ruins', 'Beni Add Cave']

Total cost: 1890.0
Daily time usage:
Day 1: 6.09 hours
Day 2: 6.23 hours
Day 3: 6.34 hours
Day 4: 6.72 hours
Day 5: 6.61 hours
Day 6: 6.19 hours
Day 7: 6.48 hours
Days used: 7


## UCS

Explores the least-cost path first using a priority queue based on path cost.

In [ ]:
import heapq
from typing import List, Dict, Tuple

def uniform_cost_search(problem: TourPlanningProblem) -> Node:
    """
    Perform Uniform Cost Search (UCS) to find an optimal itinerary.
    
    :param problem: An instance of the TourPlanningProblem.   
    :Returns:The solution Node if found, otherwise None.
    """
    # Initialize the starting node
    initial_node = Node(problem.initial_state)
    initial_node.path_cost = 0  
    initial_node.value = 0      # For priority queue
    
    # Priority queue for open nodes (using min-heap)
    open_set = []
    heapq.heappush(open_set, (initial_node.path_cost, id(initial_node), initial_node))
    
    # Dictionary to track visited states and their lowest cost
    visited = {}
    initial_state_key = state_to_key(problem.initial_state)
    visited[initial_state_key] = 0
    
    while open_set:
        # Get the node with the lowest cost
        current_cost, _, current_node = heapq.heappop(open_set)
        
        # Check if we already found a better path to this node
        current_key = state_to_key(current_node.state)
        if current_cost > visited.get(current_key, float('inf')):
            continue
            
        # Check if the current node is the goal
        if problem.is_goal(current_node.state):
            return current_node
        
        # Expand the current node
        for child in current_node.expand(problem):
            # Calculate new cost using the problem's path_cost method
            new_cost = current_node.path_cost + problem.path_cost(
                current_node.path_cost, 
                current_node.state, 
                child.action, 
                child.state
            )
            
            child_key = state_to_key(child.state)
            
            # Only consider if this is a new state or cheaper path
            if child_key not in visited or new_cost < visited[child_key]:
                visited[child_key] = new_cost
                child.path_cost = new_cost
                child.value = new_cost  # For priority queue
                heapq.heappush(open_set, (new_cost, id(child), child))
    
    return None  # No solution found

def state_to_key(state: Dict) -> Tuple:
    """
    Create a consistent, hashable key for state comparison.
    Handles floating-point GPS coordinates by rounding to 6 decimal places.
    """
    return (
        tuple(round(coord, 6) for coord in state['current_location']),  # GPS
        tuple(tuple(day) for day in state['itinerary']),  # Itinerary
        state['curr_day'],  # Current day
        round(state['total_cost'], 2),  # Total cost
        # Exclude preferences since they don't change during search
    )

### Example using UCS

In [ ]:
def run_UCS():
    constraints = {
        'max_total_budget': 5000.0,
        'max_daily_time': 7.0,
        'max_attractions_per_day': 3
    }

    user_prefs = {
        'categories': ['Museum', 'Historical']
    }

    initial_state = {
        'current_location': (35.6911, -0.6417),  # e.g., Oran coordinates
        'itinerary': [[] for _ in range(7)],
        'curr_day': 0,
        'total_cost': 0.0,
        'total_time': 0.0,
        'daily_time': [0.0] * 7,
        'preferences': user_prefs
    }

    problem = TourPlanningProblem(
        initial_state=initial_state,
        attractions=attractions_data,
        user_prefs=user_prefs,
        constraints=constraints
    )
    solution_node = uniform_cost_search(problem)
    if solution_node is None:
        print("UCS: No solution found within the depth limit.")
    else:
        print("UCS found a solution!\n")
        path_nodes = solution_node.path()
        final_state = path_nodes[-1].state

        for d, day_list in enumerate(final_state['itinerary']):
            print(f"Day {d+1}: {day_list}")

        print(f"\nTotal cost: {final_state['total_cost']}")
        print("Daily time usage:")
        for i, t in enumerate(final_state['daily_time'], 1):
            print(f"Day {i}: {t:.2f} hours")
        print(f"Days used: {final_state['curr_day']}")


if __name__ == "__main__":
    run_UCS()

### A\*
Uses a heuristic to optimize the search process by combining the current cost with the estimated cost to reach the goal.

In [ ]:
import heapq
from typing import List, Dict, Tuple

def a_star_search(problem: TourPlanningProblem) -> Node:
    """
    Perform A* search to find an optimal itinerary.
    
    Args:
        problem: An instance of TourPlanningProblem.
        
    Returns:
        The solution Node if found, otherwise None.
    """
    # Initialize the starting node
    initial_node = Node(problem.initial_state)
    initial_node.value = problem.value(initial_node.state)  # f(n) = g(n) + h(n)
    
    # Priority queue for open nodes (using min-heap)
    open_set = []
    heapq.heappush(open_set, (initial_node.value, initial_node))
    
    # Set for tracking visited states
    closed_set = set()
    
    while open_set:
        # Get the node with lowest f(n) value
        _, current_node = heapq.heappop(open_set)
        
        # Check if current node is the goal
        if problem.is_goal(current_node.state):
            return current_node
            
        # Skip if we've already processed this state
        if current_node in closed_set:
            continue
            
        closed_set.add(current_node)
        
        # Expand the current node
        for child in current_node.expand(problem):
            # Calculate the heuristic value for the child
            child.value = problem.value(child.state)
            
            # Add to open set if not already processed
            if child not in closed_set:
                heapq.heappush(open_set, (child.value, child))
    
    return None  # No solution found

### Exmaple using A* Search 

In [ ]:
def run_a_star_demo():
    # Define constraints
    constraints = {
        'max_total_budget': 5000.0,
        'max_daily_time': 7.0,
        'max_attractions_per_day': 3
    }
    
    user_prefs = {
        'categories': ['Museum', 'Historical']
    }
    
    initial_state = {
        'current_location': (35.6911, -0.6417),  # Oran coordinates
        'itinerary': [[] for _ in range(7)],
        'curr_day': 0,
        'total_cost': 0.0,
        'total_time': 0.0,
        'daily_time': [0.0] * 7,
        'preferences': user_prefs
    }
    
    problem = TourPlanningProblem(
        initial_state=initial_state,
        attractions=attractions_data,
        user_prefs=user_prefs,
        constraints=constraints
    )
    
    # Run A* search
    solution_node = a_star_search(problem)
    
    if solution_node is None:
        print("A* Search: No solution found.")
    else:
        print("A* Search found a solution!\n")
        final_state = solution_node.state
        
        for d, day_list in enumerate(final_state['itinerary']):
            print(f"Day {d+1}: {day_list}")
        
        print(f"\nTotal cost: {final_state['total_cost']}")
        print("Daily time usage:")
        for i, t in enumerate(final_state['daily_time'], 1):
            print(f"Day {i}: {t:.2f} hours")
        print(f"Days used: {final_state['curr_day']}")

if __name__ == "__main__":
    run_a_star_demo()

### Hill-Climbing
A simple greedy approach that evaluates only the immediate next state and chooses the best option available without considering future paths.

In [9]:
def hill_climbing(problem: TourPlanningProblem, initial_node: Node) -> Node:
    """
    Hill climbing search for itinerary planning.
    Iteratively improves the plan by:
    - Adding the best next attraction OR
    - Moving to the next day when stuck
    Ensures a complete 7-day itinerary.
    """
    current_node = initial_node
    current_node.value = problem.value(current_node.state)
    
    while True:
        # Generate all possible next steps (add attraction or next day)
        neighbors = current_node.expand(problem)
        if not neighbors:
            break  # No legal moves left

        # Select the best neighbor
        best_neighbor = max(neighbors, key=lambda node: problem.value(node.state))
        best_neighbor.value = problem.value(best_neighbor.state)

        # Terminate if no improvement (local maximum reached)
        if best_neighbor.value <= current_node.value:
            if current_node.state['curr_day'] < 6:
                # Force move to next day if not last day
                next_state = problem.result(current_node.state, ('next_day',))
                current_node = Node(
                    state=next_state,
                    parent=current_node,
                    action=('next_day',),
                    path_cost=current_node.path_cost
                )
                current_node.value = problem.value(next_state)
            else:
                break  # Can't improve further
        else:
            current_node = best_neighbor

    # Final completion: Ensure all days have at least 1 attraction
    final_state = current_node.state
    for day in range(7):
        if not final_state['itinerary'][day]:
            # Find first valid attraction for empty days
            for attraction in problem.attractions:
                temp_state = deepcopy(final_state)
                temp_state['curr_day'] = day
                if problem._is_valid_addition(temp_state, attraction):
                    final_state['itinerary'][day].append(attraction['name'])
                    final_state['total_cost'] += problem._parse_cost(attraction['cost'])
                    final_state['daily_time'][day] += problem._parse_duration(attraction['visit_duration'])
                    break

    current_node.state = final_state
    current_node.value = problem.value(final_state)
    return current_node

### Example using Hill Climbing

In [ ]:
def run_hill_climbing_demo():
    constraints = {
        'max_total_budget': 5000.0,  
        'max_daily_time': 7.0,       
        'max_attractions_per_day': 3 
    }

    
    user_prefs = {
        'categories': ['Historical', 'Cultural', 'Garden']  # Categories user prefS 
    }

    
    initial_state = {
        'current_location': (36.7667, 2.8833),  #  Sidi Fredj coordinates 
        'itinerary': [[] for _ in range(7)],    
        'total_cost': 0.0,                      
        'total_time': 0.0,                       
        'daily_time': [0.0] * 7,                 
        'curr_day': 0,                           
        'preferences': user_prefs                
    }

  
    problem = TourPlanningProblem(
        initial_state=initial_state,
        attractions=attractions_data,   
        user_prefs=user_prefs,          
        constraints=constraints         
    )

    
    initial_node = Node(state=initial_state)

    
    solution_node = hill_climbing(problem, initial_node)


    if solution_node is None:
        print("Hill Climbing Search: No solution found.")
    else:
        print("Hill Climbing Search found a solution!\n")
        final_state = solution_node.state

        for d, day_list in enumerate(final_state['itinerary']):
            print(f"Day {d + 1}: {day_list}")

        
        print(f"\nTotal cost: {final_state['total_cost']}")
        print("Daily time usage:")
        for i, t in enumerate(final_state['daily_time'], 1):
            print(f"Day {i}: {t:.2f} hours")
        print(f"Days used: {final_state['curr_day']+1}")



if __name__ == "__main__":
    run_hill_climbing_demo()

Hill Climbing Search found a solution!

Day 1: ['Great Mosque of Tlemcen', 'Mansourah Ruins', 'Al Mechouar Palace']
Day 2: ['Ksar Draa', "M'Zab Valley"]
Day 3: ["Bey's Palace", 'Disco Maghreb', 'Cathedral of the Sacred Heart of Oran']
Day 4: ["Arènes d'Oran", 'Fort Santa Cruz', 'Regional Theater of Oran']
Day 5: ['Villa Boulkine', "Notre Dame d'Afrique", 'Casbah of Algiers']
Day 6: ['House of the Soummam Congress (Maison du Congrès de la Soummam)', 'Roman Ruins of Tiklate (Les Ruines Romaines de Tiklate)', 'Ancient Villages Aourir and Moknea']
Day 7: ['Royal Mausoleum of Mauretania (Kbour-er-Roumia)', "La Grande Poste d'Alger", "Martyrs' Memorial (Maqam Echahid)"]

Total cost: 1040.0
Daily time usage:
Day 1: 13.70 hours
Day 2: 22.56 hours
Day 3: 15.73 hours
Day 4: 14.76 hours
Day 5: 21.21 hours
Day 6: 23.97 hours
Day 7: 21.65 hours
Days used: 7


## CSP Approach
We model the itinerary planning as a **Constraint Satisfaction Problem (CSP)**. Each attraction represents a variable, and the domain of each variable is the set of attractions that can be visited. Constraints include:
  1. **One destination per day**: Each day gets exactly one destination.
  2. **Proximity and time**: Destinations must be scheduled in a way that minimizes travel time and adheres to time constraints.
  3. **Cost**: The total cost of the trip should be within the user’s budget.
  4. **Preferences**: Ensure that the user’s preferences for types of attractions are respected.


In [ ]:
# Define constraints for the CSP
def check_constraints(itinerary, user_preferences):
    pass


## Comparative Evaluation of Algorithms 

We will compare the performance of the following search algorithms:

1. **BFS**: Guarantees the shortest path but may be slow for large datasets.
2. **DFS**: Fast but not guaranteed to find the optimal solution.
3. **A\***: Combines optimality with efficiency when the heuristic is accurate.
4. **Hill Climbing**: Greedy and fast but may get stuck in local optima.
5. **CSP Approach**: Will compare the CSP solution with the search-based methods.

We will evaluate the following metrics:
- **Execution time**: How long each algorithm takes to generate the itinerary.
- **Path optimality**: How close the solution is to the optimal itinerary in terms of cost, proximity, and user satisfaction.
- **Complexity**: The computational complexity of each approach.


In [ ]:
def evaluate_solution(itinerary, total_cost, time_taken):
    # Compare the total cost, time efficiency, and other factors
    pass  # To be implemented


## Visualizations

We will include several visualizations to help understand the results:
- **Route Map**: Visualize the itinerary on a map.
- **Cost Breakdown**: Graph showing the total cost per destination.
- **Satisfaction**: Bar chart comparing user satisfaction for different search strategies.


In [ ]:
import matplotlib.pyplot as plt

# Example plot for the cost breakdown
def plot_cost_breakdown(itinerary, attractions):
    costs = [attractions[a]['cost'] for a in itinerary]
    plt.bar(range(len(itinerary)), costs)
    plt.xlabel('Attraction')
    plt.ylabel('Cost')
    plt.title('Cost Breakdown')
    plt.show()


## Demo

In this section, we will showcase the working prototype of the **Touristic Tour Recommendation Application**. This demo will cover:
- The **interactive input** from the user (e.g., preferences, current location, etc.).
- Displaying the **optimized itinerary** generated by the selected search algorithm.
- Visualizations of the **travel route**, **cost breakdown**, and **satisfaction level**.

In [ ]:
pass

## Conclusion

In this notebook, we explored different search algorithms (A\*, Hill Climbing, BFS, DFS) and a CSP approach to generate optimal itineraries for travelers in Algeria. 
We found that **.....** performed well in terms of solution quality, but it was more computationally expensive compared to **.....**. 
Future work could involve integrating real-time weather data and optimizing routes based on current traffic conditions.
